In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

In [3]:
ratings = pd.read_csv("/home/bexx/Projects/Aristarchus/data/ratings.csv")
ratings.head()

,user_id,book_id,rating
0,7667,10572,5
1,7667,22039251,5
2,7667,1869,4
3,7667,6469165,4
4,7667,1466455,4


In [4]:
metadata = pd.read_csv("/home/bexx/Projects/Aristarchus/data/metadata.csv")
metadata = metadata.drop(['book_id:1', 'cover_url', 'Unnamed: 11', 'Unnamed: 12'], axis=1)
metadata.head()

/home/bexx/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,book_id,title,author,isbn13,published,pages,avg_rating,count,lang
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,NaN,2005,652.0,4.57,2167357.0,eng
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,9780439358071,2003,870.0,4.50,2229939.0,eng
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling,NaN,1997,309.0,4.47,6387450.0,eng
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,9780439554893,1998,352.0,4.42,6370.0,eng
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,9780439655484,1999,435.0,4.57,2438497.0,eng


In [5]:
df = pd.merge(ratings, metadata, on='book_id')
df.head()

,user_id,book_id,rating,title,author,isbn13,published,pages,avg_rating,count,lang
0,7667,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
1,19748,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
2,6784,10572,4,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
3,15758,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
4,8690,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng


In [6]:
english = df[df['lang'].str.contains('^en', regex=True, na=False)]
english['lang'].value_counts()

eng      4174414
en-US     217425
en-GB      37295
en-CA       4840
en           312
enm          188
Name: lang, dtype: int64

In [7]:
english.shape

(4434474, 11)

In [14]:
eng_books = english[english['count'] > 30000]
eng_books.head()

,user_id,book_id,rating,title,author,isbn13,published,pages,avg_rating,count,lang
0,7667,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
1,19748,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
2,6784,10572,4,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
3,15758,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
4,8690,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng


In [15]:
eng_books = eng_books[eng_books['avg_rating'] > 3.2]

### Similar books different editions

In [16]:
results = eng_books[eng_books['user_id'].map(eng_books['user_id'].value_counts()) > 11]
print(results.shape)
results.head()

(2337340, 11)


,user_id,book_id,rating,title,author,isbn13,published,pages,avg_rating,count,lang
0,7667,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
1,19748,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
2,6784,10572,4,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
3,15758,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng
4,8690,10572,5,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,9780553381696,1998,969.0,4.41,653995.0,eng


In [17]:
results['user_id'].value_counts()

3016518    162
3050756    160
3006819    158
3072333    150
3084494    150
          ... 
5551317     12
203598      12
145922      12
3089312     12
117647      12
Name: user_id, Length: 72739, dtype: int64

In [18]:
results['title'].value_counts().head(10)

Harry Potter and the Sorcerer's Stone (Harry Potter, #1)       28212
The Great Gatsby                                               24920
The Da Vinci Code (Robert Langdon, #2)                         23513
The Catcher in the Rye                                         23159
Animal Farm                                                    19497
Romeo and Juliet                                               18966
Of Mice and Men                                                18477
1984                                                           18129
Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)    17338
Harry Potter and the Goblet of Fire (Harry Potter, #4)         17336
Name: title, dtype: int64

In [19]:
results.shape

(2337340, 11)

In [20]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2337340 entries, 0 to 5277889
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     int64  
 1   book_id     int64  
 2   rating      int64  
 3   title       object 
 4   author      object 
 5   isbn13      object 
 6   published   object 
 7   pages       float64
 8   avg_rating  float64
 9   count       float64
 10  lang        object 
dtypes: float64(3), int64(3), object(5)
memory usage: 214.0+ MB


In [21]:
results.describe(include='all')

,user_id,book_id,rating,title,author,isbn13,published,pages,avg_rating,count,lang
count,2.337340e+06,2.337340e+06,2.337340e+06,2337340,2337340,1989488,2336757.0,2.337340e+06,2.337340e+06,2.337340e+06,2337340
unique,NaN,NaN,NaN,3614,1646,3249,224.0,NaN,NaN,NaN,5
top,NaN,NaN,NaN,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling,9780307277671,2005.0,NaN,NaN,NaN,eng
freq,NaN,NaN,NaN,28212,100260,23513,112362.0,NaN,NaN,NaN,2287632
mean,3.785949e+06,3.761913e+06,3.943840e+00,NaN,NaN,NaN,NaN,3.855680e+02,4.039739e+00,7.441902e+05,NaN
std,6.493878e+06,8.333640e+06,1.004376e+00,NaN,NaN,NaN,NaN,2.189454e+02,2.522904e-01,1.107781e+06,NaN
min,2.000000e+00,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,3.220000e+00,3.000600e+04,NaN
25%,2.238980e+05,5.171000e+03,3.000000e+00,NaN,NaN,NaN,NaN,2.760000e+02,3.870000e+00,8.442800e+04,NaN
50%,3.024108e+06,2.424400e+04,4.000000e+00,NaN,NaN,NaN,NaN,3.490000e+02,4.040000e+00,2.575820e+05,NaN
75%,5.555079e+06,2.213661e+06,5.000000e+00,NaN,NaN,NaN,NaN,4.520000e+02,4.210000e+00,9.936650e+05,NaN


In [22]:
results.shape

(2337340, 11)

## Some cleaning

In [23]:
results.isnull().sum()

user_id            0
book_id            0
rating             0
title              0
author             0
isbn13        347852
published        583
pages              0
avg_rating         0
count              0
lang               0
dtype: int64

# Utility Matrix

In [24]:
utility_matrix = results.pivot_table(values='rating', index='user_id',
                                       columns='title', fill_value=0)

In [25]:
utility_matrix.head()

title,#GIRLBOSS,'Salem's Lot,'Tis,"1,000 Places to See Before You Die","10% Happier: How I Tamed the Voice in My Head, Reduced Stress Without Losing My Edge, and Found Self-Help That Actually Works","10th Anniversary (Women's Murder Club, #10)",11/22/63,12 Rules for Life: An Antidote to Chaos,"13 Little Blue Envelopes (Little Blue Envelope, #1)",1491: New Revelations of the Americas Before Columbus,...,"You (You, #1)","You Are Special (Wemmicksville, #1)",You Can Heal Your Life,"You Suck (A Love Story, #2)",You Will Know Me,Z: A Novel of Zelda Fitzgerald,"Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values (Phaedrus, #1)","Zero Day (John Puller, #1)","Zero to One: Notes on Startups, or How to Build the Future",Zoo (Zoo #1)
user_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
utility_matrix.shape

(72739, 3614)

In [27]:
Arranged = utility_matrix.T

In [28]:
SVD = TruncatedSVD(n_components=100)
transposed_matrix = SVD.fit_transform(Arranged)
transposed_matrix.shape

(3614, 100)

In [29]:
corr_matrix = np.corrcoef(transposed_matrix)

books = utility_matrix.columns
books_list = list(books)

In [123]:
search = 'Harry Potter and the Order of the Phoenix'
print(search.title())

Harry Potter And The Order Of The Phoenix


In [125]:
import re

In [124]:
book_01 = books_list.index(search.title())
book_01

ValueError: 'Harry Potter And The Order Of The Phoenix' is not in list

In [110]:
corr_01 = corr_matrix[book_01]

In [119]:
rec = list(books[(corr_01 < 1.0) & (corr_01 >= 0.5)])[:5]

In [120]:
rec

['Animal Farm', 'Animal Farm / 1984', 'Brave New World']

In [93]:
db = results.drop_duplicates(subset=['book_id'])
db = db[['title', 'author', 'published', 'pages', 'avg_rating', 'count']]
db['pages'] = db['pages'].astype(int)
db.head()

,title,author,published,pages,avg_rating,count
0,"A Clash of Kings (A Song of Ice and Fire, #2)",George R.R. Martin,1998,969,4.41,653995.0
3722,Nickel and Dimed: On (Not) Getting by in America,Barbara Ehrenreich,2001,240,3.63,169675.0
7559,What Alice Forgot,Liane Moriarty,2009,476,4.08,270094.0
9183,The Last Anniversary,Liane Moriarty,2006,388,3.75,59925.0
9551,"The Magicians (The Magicians, #1)",Lev Grossman,2009,402,3.51,195511.0


In [94]:
for title in rec:
    print(db[df['title'] == title])

/home/bexx/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/bexx/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


                  title               author published  pages  avg_rating  \
2470408  As You Like It  William Shakespeare      1599    263        3.83   

           count  
2470408  63271.0  
               title               author published  pages  avg_rating  \
2841406  Dr. Faustus  Christopher Marlowe      1589     64         3.8   

           count  
2841406  46437.0  


/home/bexx/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/bexx/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


           title               author published  pages  avg_rating    count
3032483  Henry V  William Shakespeare      1600    294        3.88  35694.0
            title               author published  pages  avg_rating     count
988746  King Lear  William Shakespeare      1605    338        3.91  157837.0
                          title               author published  pages  \
1961384  Much Ado About Nothing  William Shakespeare      1598    246   

         avg_rating     count  
1961384        4.07  196217.0  


/home/bexx/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
